# Desafío 01

## Integrantes

- Acevedo Zain, Gaspar (acevedo.zain.gaspar@gmail.com)

## Consignas

**Cada experimento realizado debe estar acompañado de una explicación o interpretación de lo observado.**

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Construir un modelo de clasificación por prototipos (tipo zero-shot). Clasificar los documentos de un conjunto de test comparando cada uno con todos los de entrenamiento y asignar la clase al label del documento del conjunto de entrenamiento con mayor similaridad.

**3**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**NO cambiar el hiperparámetro ngram_range de los vectorizadores**.

**4**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares.

**Elegir las palabras MANUALMENTE para evitar la aparición de términos poco interpretables**.


## Imports y carga de datos

In [1]:
%pip install numpy scikit-learn

In [68]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

from sklearn.datasets import fetch_20newsgroups
import numpy as np

from scipy.sparse import csr_matrix
from typing import Tuple

import random

In [4]:
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Resolución ejercicio 1

***Enunciado***

- Vectorizar documentos.
- Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
- Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

Comenzamos inicializando un vectorizador del tipo [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) con los parámetros por defecto.

In [11]:
tfidVectorizer_01 = TfidfVectorizer()

Una vez inicializado el vectorizador, se procede a vectorizar el set de entrenamiento (`newsgroups_train`).

In [12]:
X_train = tfidVectorizer_01.fit_transform(newsgroups_train.data)

Se muestra que hay $11314$ documentos vectorizados en el set de entrenamiento.

In [16]:
cant_docs = X_train.shape[0]
print(f"Cantidad de documentos en train: {cant_docs}")

Cantidad de documentos en train: 11314


A continuación, se seleccionan al azar $5$ documentos vectorizados del set de entrenamiento.

In [17]:
random_idxs = [random.randint(0, cant_docs -1) for _ in range(5)]

In [48]:
corpus = newsgroups_train

Se definen dos funciones de utilidad:

- `get_documents_similarity`, la cual, dados una `matriz documento-término` y un índice de un documento de la misma, obtiene la `similitud coseno` entre dicho documento y el resto de los documentos de la matriz, como así también los índices de estos documentos.
- `get_k_most_similar_documents`, la cual utiliza la función anterior, y devuelve a los $K$ documentos con mayor `similitud coseno` junto a sus índices. Por defecto $K = 5$.

In [72]:
def get_documents_similarity(matriz_dt: csr_matrix, idx: int) -> Tuple[np.ndarray, np.ndarray]:
  """
  Devuelve la similitud coseno de un documento respecto al resto de los
    documentos de una matriz documento-término.

  Args:
    matriz_dt (csr_matrix): Matrix documento-término a partir de la cual se
      obtendrá la similitud coseno de un documento, dado su índice.
    idx (int): índice del documento sobre el cual se trabajará.
  Returns:
    Tuple[np.ndarray, np.ndarray]:
      Elem 0: documentos ordenados de mayor a menor similitud coseno.
      Elem 1: argumentos correspondientes a Elem 0.
  Raises:
    IndexError, si el parámetro `idx` está fuera del rando de índices de
      `matriz_dt`.
  """

  if idx >= matriz_dt.shape[0]:
    raise IndexError(f"El parámetro `idx` está fuera del rango del corpus \
    ({matriz_dt.shape[0]}).")

  documents_similarity = cosine_similarity(matriz_dt[idx], matriz_dt)[0]

  sorted_docs = np.sort(documents_similarity)[::-1]
  arg_docs = np.argsort(documents_similarity)[::-1]
  return sorted_docs, arg_docs


In [73]:
def get_k_most_similar_documents(matriz_dt: csr_matrix, idx: int, K: int = 5) -> Tuple[np.ndarray, np.ndarray]:
  """
  Dado una matríz documento-término y un índice correspondiente a uno de sus documentos, devuelve
  los K documentos con mayor similitud coseno respecto del documento indicado.

  Args:
    matriz_dt (csr_matrix): Matrix documento-término a partir de la cual se
      obtendrá la similitud coseno de un documento, dado su índice.
    idx (int): índice del documento sobre el cual se trabajará.
    K (int): número de documentos con mayor similitud que se quiere encontrar.
      Por defecto es 5.
  Returns:
    np.ndarray: array de longitud K que contiene la similitud coseno del
      documento a analizar respecto al resto, ordenados de mayor a menor.
    Tuple[np.ndarray, np.ndarray]:
      Elem 0: `K` documentos ordenados de mayor a menor similitud coseno.
      Elem 1: argumentos correspondientes a Elem 0.
  Raises:
    IndexError, si el parámetro `idx` está fuera del rando de índices de
      `matriz_dt`.
  """

  if idx >= matriz_dt.shape[0]:
    raise IndexError(f"El parámetro `idx` está fuera del rango de la matriz \
    documento-término ({matriz_dt.shape[0]}).")

  docs_similarity, docs_args  = get_documents_similarity(matriz_dt=matriz_dt, idx=idx)

  # El índice 0 corresponde al propio documento
  return docs_similarity[1:K + 1], docs_args[1:K + 1]

### Documento con índice $11097$

In [49]:
idx = random_idxs[0]
print(idx)

11097


In [57]:
target_data = corpus.data[idx]

target_idx = corpus.target[idx]
target_type = corpus.target_names[target_idx]

In [55]:
print(target_data)


I have also heard it called an expression of mercy, because Heaven would be
far more agonizing for those who had rejected God.



In [58]:
print(target_type)

soc.religion.christian


In [74]:
a, b = get_k_most_similar_documents(matriz_dt=X_train, idx=idx)

In [76]:
a

array([0.26512275, 0.20356234, 0.20273585, 0.2021826 , 0.17593852])

## Resolución ejercicio 2

## Resolución ejercicio 3

## Resolución ejercicio 4